In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

from lib.data.datareader_total import DataReaderTotal
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.utils.args import get_opts_args, get_opt_args_from_model_name
from lib.model.load_model import load_model
from lib.model.load_dataset import load_dataset
from lib.data.dataset_motion_3d import MotionDataset3DTotal
from lib.model.evaluation import *

### Evaluate single experiment

In [2]:
checkpoint = 'MB_train_h36m_gt'
args, opts = get_opt_args_from_model_name(checkpoint)
model_pos, chk_filename, checkpoint = load_model(opts, args)

subset = 'FIT3D-GT-ALL_TEST'
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)

# if '3DHP' in subset:
#     args.eval_part = 'whole_without_nose'
#     args.part_list = ['whole_without_nose']

configs/pose3d/MB_train_h36m_gt.yaml
True
MB_train_h36m_gt
Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt/best_epoch.bin


In [6]:
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

INFO: Testing
No epoch information in the checkpoint


  0%|          | 0/411 [00:00<?, ?it/s]

100%|██████████| 411/411 [04:14<00:00,  1.61it/s]


No eval_keypoint. Use part list
Part: whole
+------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------------+------------------------+-----------------------+---------------------+----------------------------------+------------------------+--------------------+--------------------+--------------------+---------------------------------+-------------------+-----------------------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------

In [17]:
save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{args.model}'
save_path = os.path.join(save_folder, f'{subset}.pkl')
print(save_path)
results_dict = {'results_all': results_all, 'gts_all': gts_all, 'inputs_all': inputs_all, 'total_result_dict': total_result_dict}
savepkl(results_dict, save_path)

/home/hrai/codes/MotionBERT/saved_results/MB_train_h36m_gt/FIT3D-GT-ALL_TEST.pkl


### Evaluate and save all experiments

### Cherrypick e1, e1 -> result_dict.pkl

In [9]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']


if os.path.exists('result_dict.pkl'):
    print('Loading result_dict.pkl')
    result_dict = readpkl('result_dict.pkl')
else:
    result_dict = {}
result_root = '/home/hrai/codes/MotionBERT/saved_results'
checkpoint_list = os.listdir(result_root)
for checkpoint in tqdm(checkpoint_list):
    if checkpoint in blacklist_checkpoint: continue # skip
    if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
    checkpoint_root = os.path.join(result_root, checkpoint)
    subset_list = glob(checkpoint_root+'/*.pkl')
    for item in subset_list:
        subset = os.path.basename(item).split('.')[0]
        if subset in result_dict[checkpoint].keys(): 
            #print(f'{subset} exists')
            continue
        result = readpkl(item)
        result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}

Loading result_dict.pkl


100%|██████████| 24/24 [00:02<00:00,  9.26it/s]


In [10]:
savepkl(result_dict, 'result_dict.pkl')